# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [359]:
import pandas as pd
import numpy as np
import math as math
from scipy import stats
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords

Renomear categorias

In [360]:
tweets = pd.read_excel('tweets.xlsx')

tweets = tweets.replace(0,"Irrelevante")
tweets = tweets.replace(2,"Positivo")
tweets = tweets.replace(1,"Negativo")

20


Excluir caracteres específicos

In [361]:
tweets["Treinamento"] = tweets["Treinamento"].apply(lambda x: x.lower())
# tweets["Treinamento"] = tweets["Treinamento"].apply(lambda x: x.replace("café"," "))
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("rt", ",")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace('"',"")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace('“',"")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("'","")    
# tweets["Treinamento"] = tweets["Treinamento"].str.replace(",","")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace(" :","")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace(": ","")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace(" : ","")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace(".","")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("(","")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace(")","")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace(" é "," ")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace(" á "," ")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace(" à "," ")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace(" ao "," ")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace(" à "," ")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace(" a "," ")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace(" o "," ")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("é ","")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("á ","")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("à","")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("ao ","")

# for frase in tweets["Treinamento"]:
#     frase_split = frase.split(" ")
#     for palavra in frase_split:
#         if len(palavra) == 1:
#             frase_split.remove(palavra)
            
    
nltk.download('stopwords')
stop = stopwords.words('portuguese')
stop_list = ["?", "de", "é", "á", "à", "ao", "a", "o", "é", "rt", '"', "“", "'", ",", ":", ".", "(", ")", "!", "$", "%", "*", "&", "-", "+", "=", "/"]
stop.extend(stop_list)
tweets['Treinamento'] = [' '.join([w for w in x.lower().split() if w not in stop]) 
    for x in tweets['Treinamento'].tolist()]

            
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("a "," ")

# tweets["Treinamento"] = tweets["Treinamento"].str.replace("o ","")


# tweets["Treinamento"] = tweets["Treinamento"].str.replace("!"," ")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("$"," ")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("%"," ")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("*","")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("&","")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("-","")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("+","")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("=","")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace("/ "," ")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace(" /"," ")
# tweets["Treinamento"] = tweets["Treinamento"].str.replace(" / "," ")

tweets["Treinamento"] = tweets["Treinamento"].str.replace("  "," ")
tweets

[nltk_data] Downloading package stopwords to C:\Users\Andre
[nltk_data]     Weber\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Treinamento,Categoria
0,falei pra irmã pra acelerar café ia tomar yaku...,Positivo
1,gringo emocionando yakult kskskksksksksksksks ...,Negativo
2,fico eh mto chocada palavra yakult disseminada...,Positivo
3,diga nesse cc deixa irritada — tomar yakult ht...,Irrelevante
4,@diih_guilbert yakult horrível,Negativo
5,assim apaixonei yakult https://t.co/ohsmrx2zsl,Positivo
6,@anarcomuna acertei dessa vez. drink lá yakult...,Positivo
7,pq ontem bebi mt yakult filho novinha kkk,Positivo
8,@renanzinhaa peter yakult passo fim semana,Irrelevante
9,tipo mercado pra comprar essencial p semana co...,Positivo


In [362]:
teste = tweets['Treinamento'].str.count('falei') + 1

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


In [363]:
dic = []

for tweet in tweets["Treinamento"]:
    for palavra in tweet.split(" "):
        if palavra not in dic:
            dic.append(palavra)
            
dic = {"Palavras": dic}
dic = pd.DataFrame(dic)
dic

,Palavras
0,falei
1,pra
2,irmã
3,acelerar
4,café
5,ia
6,tomar
7,yakult
8,"dela,"
9,aí
